## 0. Libarary 불러오기 및 경로설정

In [175]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from pandas import DataFrame
import torch.optim as optim

In [176]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
train_dir = '/opt/ml/input/data/train'
test_dir = '/opt/ml/input/data/eval'

## 1. Model 정의

In [177]:
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 1000):
        super(MyModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## 2. Train Dataset 정의

In [204]:
class TrainDataset(Dataset):    #Dataset만 받아야 한다.
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform
        set.x = Dataset.image
        self.y = Dataset.mask,Dataset.gender,Dataset.age

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

## 2. Test Dataset 정의

In [179]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [180]:
model = MyModel(1000)
loss = nn.CrossEntropyLoss()
optm = optim.Adam(model.parameters(), lr=1e-3)
print("Done.")

Done.


## 3. Pre-Processing

In [200]:
train_csv= pd.read_csv(os.path.join(train_dir, 'train.csv'))

tmp_x_paths = [os.path.join(train_dir,img_id) for img_id in train_csv.path]

train_data = DataFrame({"image":[], "mask":[], "gender":[], "age":[]})

names = ['incorrect_mask.jpg','mask1.jpg','mask2.jpg','mask3.jpg','mask4.jpg','mask5.jpg','normal.jpg']
for i,img_dir in enumerate(tmp_x_paths):
    for j, name in enumerate(names):
        if name[0]=='m':
            train_data.loc[7*i+j] = [os.path.join(img_dir,name), 0, train_csv.gender[i],train_csv.age[i]]
        elif name[0]=='i':
            train_data.loc[7*i+j] = [os.path.join(img_dir,name), 1, train_csv.gender[i],train_csv.age[i]]
        else:
            train_data.loc[7*i+j] = [os.path.join(img_dir,name), 2, train_csv.gender[i],train_csv.age[i]]


In [208]:
train_data

,image,mask,gender,age
0,/opt/ml/input/data/train/000001_female_Asian_4...,1.0,female,45.0
1,/opt/ml/input/data/train/000001_female_Asian_4...,0.0,female,45.0
2,/opt/ml/input/data/train/000001_female_Asian_4...,0.0,female,45.0
3,/opt/ml/input/data/train/000001_female_Asian_4...,0.0,female,45.0
4,/opt/ml/input/data/train/000001_female_Asian_4...,0.0,female,45.0
...,...,...,...,...
18895,/opt/ml/input/data/train/006959_male_Asian_19/...,0.0,male,19.0
18896,/opt/ml/input/data/train/006959_male_Asian_19/...,0.0,male,19.0
18897,/opt/ml/input/data/train/006959_male_Asian_19/...,0.0,male,19.0
18898,/opt/ml/input/data/train/006959_male_Asian_19/...,0.0,male,19.0


In [207]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

x_data = TrainDataset(train_data,transform)

train_loader = DataLoader(x_data, shuffle=True)

AttributeError: type object 'Dataset' has no attribute 'image'

## 3. Inference

In [26]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
testImage_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
testImage_paths = [os.path.join(testImage_dir, img_id) for img_id in submission.ImageID]




transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(testImage_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = MyModel(num_classes=18).to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
